<a href="https://colab.research.google.com/github/springboardmentor0327/Text_Summarization_Infosys_Internship_Oct2024/blob/BandariRohith/Week_2_Abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Necessary Libraries

In [3]:
# Importing necessary libraries
!pip install transformers datasets rouge-score nltk
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c85cc7dfa6e3162c34ea38be6b7d363ce0d9a91e85f03dc46128cef95136472e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Setting up environment variables for API authentication

In [5]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('API')

Loading the Datasets

In [6]:
# Load CNN DailyMail Dataset
dataset = load_dataset("cnn_dailymail", '3.0.0', split='test')
print(dataset[0])  # Print the first item in the test dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

Loading Pre-trained BART and T5 Models and LLM



For BART

In [7]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

For T5

In [8]:
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


For LLM

In [9]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 10.8 MB/s eta 0:00:00


In [10]:
  from langchain_google_genai import ChatGoogleGenerativeAI
  # from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

  def load_llm(model="gemini-1.5-pro"):

    if model == "gemini-1.5-pro":
      llm = ChatGoogleGenerativeAI(
          model="gemini-1.5-pro",
          temperature=0,
          max_tokens=None,
          timeout=None,
          max_retries=2)
      return llm
    elif model == "gemini-1.5-flash":
      llm = ChatGoogleGenerativeAI(
          model="gemini-1.5-flash",
          temperature=0,
          max_tokens=None,
          timeout=None,
          max_retries=2)
      return llm
    else:
      raise ValueError("Invalid model name")

Summarization Functions

BART Summarization Function:

In [11]:
def summarize_bart(text):
    inputs = bart_tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

T5 Summarization Function:

In [12]:
def summarize_t5(text):
    inputs = t5_tokenizer("summarize: " + text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

LLM Summarization Function:

In [13]:
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate


def get_prompt_template():
  # Define prompt
  prompt = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              "Write a concise summary of the following in {num_words} words:\\n\\n",
          ),
          ("human", "{context}")
      ]
  )

  return prompt


def summarize_text(text, num_words=50, model="gemini-1.5-pro"):

  # Load LLM
  llm = load_llm(model)

  # Get Prompt Template
  prompt = get_prompt_template()

  # Instantiate chain
  chain = prompt | llm

  # Invoke chain
  result = chain.invoke({
        "context": text,
        "num_words": num_words
      })

  # Return result
  return result.content

Testing on CNN/DailyMail Dataset

In [14]:
# Take a small sample of the dataset
sample = dataset.select([0])  # Select a specific article for demo

# Get the original text and summary
original_text = sample['article'][0]
reference_summary = sample['highlights'][0]

# Generate BART and T5 summaries
bart_summary = summarize_bart(original_text)
t5_summary = summarize_t5(original_text)
llm_summary = summarize_text(original_text, num_words = 50, model = "gemini-1.5-flash")


print("BART Summary:\n", bart_summary)
print("T5 Summary:\n", t5_summary)
print("LLM Summary:\n", llm_summary)
print("Reference Summary:\n", reference_summary)



BART Summary:
 The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki said it was a move toward greater justice.
T5 Summary:
 the formal accession was marked by a ceremony at The Hague, in the Netherlands . the ICC opened a preliminary examination into the situation in the occupied territories . as members of the court, Palestinians may be subject to counter-charges .
LLM Summary:
 Palestine officially joined the International Criminal Court (ICC), granting the court jurisdiction over alleged crimes in Palestinian territories since June 2014.  This move, opposed by Israel and the US, allows for potential war crimes investigations against both sides, furthering the pursuit of justice and peace, though preliminary.

Reference Summary:
 Membershi